<img src="../logo.png" height="200" width="900"> 

# <center> Сбор данных: полезные хитрости </center>

### Хитрость 1:  Не стесняйтесь пользоваться `try-except`

Эта конструкция позволяет питону в случае ошибки сделать что-нибудь другое либо проигнорировать её. Например, мы хотим найти логарифм от всех чисел из списка: 

In [1]:
from math import log 

a = [1,2,3,-1,-5,10,3]

for item in a:
    print(log(item))

0.0
0.6931471805599453
1.0986122886681098


ValueError: math domain error

У нас не выходит, так как логарифм от отрицательных чисел не берётся. Чтобы код не падал при возникновении ошибки, мы можем его немного изменить: 

In [5]:
from math import log 

a = [1,2,3,-1,-5,10,3]

for item in a:
    try:
        print(log(item)/0)  # попробуй взять логарифм
    except:
        print('я не смог') # если не вышло, сознайся и работай дальше

я не смог
я не смог
я не смог
я не смог
я не смог
я не смог
я не смог


__Как это использовать при парсинге?__  Интернет создаёт человек. Он часто ошибается. Предположим, что мы на ночь поставили парсер скачивать цены, он отработал час и упал из-за того, что на како-нибудь одной странице были неверно проставлены теги, либо вылезло какое-то редкое поле, либо вылезли какие-то артефакты от старой версии сайта, которые не были учтены в нашем парсере. Гораздо лучше, чтобы код проигнорировал эту ошибку и продолжил работать дальше. 

### Хитрость 2:  pd.read_html

Если на странице, которую вы спарсили, среди тэгов `<tr>` и `<td>` прячется таблица, чаще всего можно забрать её себе без написания цикла, который будет перебирать все стобцы и строки. Поможет в этом `pd.read_html`. Например, вот так можно забрать себе [табличку с сайта ЦБ](https://cbr.ru/currency_base/daily/) 

In [7]:
import pandas as pd

dfs = pd.read_html('https://cbr.ru/currency_base/daily/', header=-1)
dfs[0].head()

,Цифр. код,Букв. код,Единиц,Валюта,Курс
0,36,AUD,1,Австралийский доллар,522386
1,944,AZN,1,Азербайджанский манат,431048
2,51,AMD,100,Армянских драмов,150938
3,933,BYN,1,Белорусский рубль,297885
4,975,BGN,1,Болгарский лев,440247


Команда пытается собрать в массив все таблички c веб-страницы. Если хочется, можно сначала через bs4 найти нужную таблицу, а потом уже распарсить её: 

In [8]:
import requests
from bs4 import BeautifulSoup

resp = requests.get('https://cbr.ru/currency_base/daily/')
tree = BeautifulSoup(resp.content, 'html.parser')

# нашли табличку
table = tree.find_all('table', {'class' : 'data'})[0]

# распарсили её
df = pd.read_html(str(table), header=-1)[0]
df.head()

,Цифр. код,Букв. код,Единиц,Валюта,Курс
0,36,AUD,1,Австралийский доллар,522386
1,944,AZN,1,Азербайджанский манат,431048
2,51,AMD,100,Армянских драмов,150938
3,933,BYN,1,Белорусский рубль,297885
4,975,BGN,1,Болгарский лев,440247


### Хитрость 3:  используйте пакет tqdm

> Код уже работает час. Я вообще без понятия когда он закончит работу. Было бы круто узнать, сколько ещё ждать... 

Если в вашей голове возникла такая мысль, пакет `tqdm` ваш лучший друг. Установите его: ```pip install tqdm```

In [10]:
from tqdm import tqdm_notebook
import time

a = list(range(30))

# 30 раз будем спать по секунде
for i in tqdm_notebook(a):
    time.sleep(1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


Мы добавили тот вектор, по которому идёт цикл в `tqdm_notebook`. Это даёт нам красивую зелёную строку, которая показывает, насколько сильно мы продвинулись по коду.

### Хитрость 4:  распаралеливание

Если сервер не очень настроен вас банить, можно распаралелить свои запросы к нему. Самый простой способ сделать это — библиотека `joblib`. 

In [12]:
!pip install joblib

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.2.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [13]:
from joblib import Parallel, delayed
from tqdm import tqdm_notebook

def simple_function(x):
    return x**2

nj = -1 # паралель на все ядра 
result = Parallel(n_jobs=nj)(
                delayed(simple_function)(item)          # какую функцию применяем 
                for item in tqdm_notebook(range(10)))   # к каким объектам применям

# tqdm_notebook в последней строчке будет создавать зелёный бегунок с прогрессом

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


На самом деле это не самый эффективный способ паралелить в python, но зато он занимает две строчки.

### Хитрость 5:  selenium без браузера

Селениум можно настроить так, чтобы физически браузер не открывался.

In [56]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

options = Options()
options.headless = True 
driver = webdriver.Firefox(options=options)

ref = 'http://google.com'
driver.get(ref)

driver.close()

###  Ещё хитрости: 

* __Сохраняйте то, что парсите по мере скачки!__ Прямо внутрь цикла добавьте код, который сохраняет файл! 
* Когда код упал в середине списка для скачки, не обязательно запускать его с самого начала. Просто сохраните тот кусок, который уже скачался и дозапустите код с места падения.
* Добавлять цикл для обхода ссылок внутрь функции - не самая хорошая идея. Предположим, что надо обойти $100$ ссылок. Функция должна вернуть на выход объекты, которые скачались. Она берёт и падает на $50$ объекте. Конечно же то, что уже было скачано, функция не возвращает. Всё, что вы накачали - вы теряете. Надо запускать заново. Почему? Потому что внутри функции своё пространство имён. Если бы вы делали это циклом без функции, то можно было бы сохранить первые $50$ объектов, которые уже лежат внутри листа, а потом продолжить скачку. 
* Можно ориентироваться на html-страничке с помощью `xpath`. Он предназначен для того, чтобы внутри html-странички можно было быстро находить какие-то элементы. [Подробнее можно почитать тут.](https://devhints.io/xpath)
* Не ленитесь читать документацию. Из неё можно узнать много полезного. 